In [1]:
# Kütüphanelerin Aktarılması
import sys
import threading
import datetime
import time
import os
import pickle
import glob
import contextlib
import numpy as np
from shutil import copyfile

from PyQt5 import QtWidgets
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QDir,QTimer,Qt
from PyQt5.QtChart import QChart, QChartView, QBarSet, QBarSeries, QBarCategoryAxis, QValueAxis

from anasayfa import *
from kayitekrani import *
from yukleniyor import *
from analiz import *
from dialogUyari import *
from dialogKaydet import *

import noisereduce as nr
import librosa
import sounddevice as sd
from scipy.io.wavfile import write,read

import ctypes
myappid = 'IGU.bitirmeprojesi.2020'
ctypes.windll.shell32.SetCurrentProcessExplicitAppUserModelID(myappid)

C:\Users\Omer\Anaconda3\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


0

In [2]:
# Global değişkenlerin tanımlanması
file_path = None
isRecPart = None
myrecording=None
timer = None
timer2 = None
event = None
predY=[]

# Bu kısım diğer bilgisayarlarda uygun bir şekilde değiştirilmeli
MODEL_PATH = 'C:\\Users\\Omer\\Desktop\\Dersler\\Proje\\Emotion_Recognition\\Models\\modelMLP.sav'

situation="Record"
clock = '00:00:00'

count = 0
labeltexts = [
    "Seste varolan gürültüler gideriliyor ...",
    "Sesin özellik çıkarımları gerçekleştiriliyor ...",
    "Modeller hazırlanıyor ...",
    "Ses analiz ediliyor ...",
    "Sonuçlar Hazır !"
]
texts = [
    "Dosya Adı : ",
    "Tarih : ",
    "Verilen Asıl Duygu : ",
    "Kullanılan Model : ",
    "Sonuçlar : \n"
]
emotions = [
    "Normal",
    "Sakin",
    "Mutlu",
    "Üzgün",
    "Sinirli"
]

In [3]:
# Pencere ve Dialogların oluşturulması

app = QApplication(sys.argv)

#Main Window
mainWindow = QMainWindow()
ui =Ui_MainWindow()
ui.setupUi(mainWindow)

#Record Window
recordWindow = QWidget()
ui2 = Ui_Kayit_Ekrani()
ui2.setupUi(recordWindow)

#Analysis Window
analysisWindow = QWidget()
ui4 = Ui_ERForm()
ui4.setupUi(analysisWindow)

#Loading Dialog
loading_dialog = QDialog()
ui3 = Ui_Dialog()
ui3.setupUi(loading_dialog)

# Warning Dialog
uyariDialog = QDialog()
ui5 = Ui_DialogUyari()
ui5.setupUi(uyariDialog)

# Save Dialog
save_dialog = QDialog()
ui6 = Ui_KaydetDialog()
ui6.setupUi(save_dialog)

In [4]:
# Kayıt ekranının açılması için tanımlanan fonksiyon
def openRW(): # Record Window 
    global isRecPart
    recordWindow.show()
    situation = "Record"
    isRecPart=True

In [5]:
# Dosya seçmek için tanımlanan fonksiyon
def openFileWindow():
    global isRecPart,file_path
    isRecPart=False
    # Dosya seçme penceresi için
    # Başlangıç yolu olarak C diskini seçtik ve sadece wav dosyalarını göstermesini istedik
    file_path = QFileDialog.getOpenFileName(None, 'Bir wav dosyası seçiniz', 
   'C:\\',"Wav Files (*.wav)") 
    file_path = file_path[0]
    filename = file_path.split("/") # Dosya yolunu '/' karakterlerini baz alarak bölüp bir diziye atadık 
    filename=filename[-1] # Dosya ismimiz son kısımda olduğundan dolayı -1. indeksi aldık
    
    # Uyarı verdik
    ui5.uyari_lbl.setText("Seçtiğiniz \""+filename+"\" dosyasının sadece ilk 5 saniyesi kullanılacaktır\n"+
    "Onaylıyor musunuz ?")
    ui5.onay_btn.clicked.connect(lambda: openLW(file_path))
    if file_path != "":
        uyariDialog.show()

In [6]:
# Kayıt ekranında kayıt , durduma ve oynatmak işlemlerini tanımladık
# situation adındaki global değişken ile durumları hafızada tuttuk
def RPP(): # Record Play Pause
    global situation
    if situation in "Record":
        ui2.record_btn.setStyleSheet("border-radius : 75;\n"
                                     "image: url(:/newPrefix/stop.png);")
        situation = "Paused"
        record()
        
    elif situation in "Paused":
        stop()
    elif situation in "Play":
        play()

In [7]:
# Ses kaydı için tanımlanan fonksiyon
# myrecording global değişkeni ile kaydettiğimiz dosyanın her yerden erişilebilirliği sağladık
def record():
    global myrecording
    start_timer()
    seconds = 5
    myrecording = sd.rec(int(seconds * 44100), samplerate=44100, channels=1,blocking=False)
    

In [8]:
# Kaydı oynatma için tanımlanan fonksiyon
def play():
    global myrecording
    sd.play(myrecording,44100)

In [9]:
# Ses kaydının durdurulması için tanımlanan fonksiyon
def stop():
    global myrecording,situation
    sd.stop()
    stop_timer()
    write('output.wav', 44100, myrecording)  # Save as WAV file 
    ui2.record_btn.setStyleSheet("border-radius : 75;\n"
                                     "image: url(:/newPrefix/play-button.png);")
    situation = "Play"
    # Silme ve Onaylama butonlarını aktifleştirdik
    ui2.delete_btn.setEnabled(True)
    ui2.okey_btn.setEnabled(True)

In [10]:
# Ses kaydının beğenilmemesi durumunda silinmesi için tanımlanan fonksiyon
def deleteSound():
    global situation
    # Silinmesi durumunda zaman kısmının tekrar başlanımı
    ui2.time_label.setText('00:00:00')
    ui2.record_btn.setStyleSheet("border-radius : 75;\n"
                                 "image: url(:/newPrefix/radio-microphone.png);")
    situation="Record"
    # Silme ve Onaylama butonlarını deaktif ettik
    ui2.delete_btn.setEnabled(False)
    ui2.okey_btn.setEnabled(False)

In [11]:
# Zamanın başlatılması işlemi
# Threadlar yardımı ile kontrolünü yaptık
def start_timer():
    global timer,event
    print('start')
    timer = threading.Thread(target=get_time)
    event = threading.Event()
    timer.daemon=True
    timer.start() 

In [12]:
# Zaman arttırımı işlemini bu fonksiyon içerisinde yaptık
def get_time():
    global event,clock
    seconds = 0
    while not event.is_set():
        seconds += 1
        clock = str(datetime.timedelta(seconds = seconds))
        ui2.time_label.setText(clock)
        time.sleep(1)
        if seconds == 5:
            stop()

In [13]:
# Zamanlayıcıyı durdumak için tanımlanan fonksiyon 
def stop_timer():
    global event
    print('stop')
    event.set()
    timer = None

In [14]:
# Sesi onayladıktan sonra açılan yükleme ekranı için tanımlanan fonksiyon
def openLW(file_path="output.wav"): # LW = Loading Window
    global isRecPart
    # Bu global değişkende kayıt ekranında isek uyarının gösterilmemesi amaçlanmıştır
    if not isRecPart:
        uyariDialog.hide()
    global loading_dialog,ui3,count
    count=0
    loading_dialog = QDialog()
    ui3.setupUi(loading_dialog)
    ui3.sonuc_btn.clicked.connect(lambda: openAW(file_path))
    loading_dialog.show()
    global timer2
    timer2 = QTimer()
    timer2.timeout.connect(increaseProgress)
    timer2.start(1000)

In [15]:
#Progress Bar'ın ilerlemesi için tanımlan fonksiyon
def increaseProgress():
    global count,labeltexts
    value = ui3.progressBar.value()
    if value < 100:
        value = value + 20
        ui3.progressBar.setValue(value)
        
        ui3.info_label.setText(labeltexts[count])
        count+=1
    else:
        ui3.sonuc_btn.setEnabled(True)
        timer2.stop()

In [16]:
# Analiz Ekranının gelen verilerle oluşturulması için tanımlanan fonksiyon
def openAW(file_path="output.wav"): # Analysis Window
    global analysisWindow,ui4,emotions,predY
    analysisWindow = QWidget()
    ui4 = Ui_ERForm()
    ui4.setupUi(analysisWindow)
    ui4.kaydet_btn.clicked.connect(openSD)
    
    loading_dialog.hide()
    analysisWindow.show()
    loaded_model = pickle.load(open(MODEL_PATH, 'rb'))
    print(file_path)
    for file in glob.glob(file_path):
        feature = np.array(extract_feature(file))
        predY = loaded_model.predict_proba([feature])
    print(predY)
    
    if file_path in "output.wav":
        ui4.filename_lbl.setText(file_path)
    else:
        ui4.filename_lbl.setText(file_path.split("/")[-1])
        
    
    ui4.normal_lbl.setText(toPercantage(predY[0][0]))
    ui4.sakin_lbl.setText(toPercantage(predY[0][1]))
    ui4.mutlu_lbl.setText(toPercantage(predY[0][2]))
    ui4.uzgun_lbl.setText(toPercantage(predY[0][3]))
    ui4.sinirli_lbl.setText(toPercantage(predY[0][4]))
    
    set0 = QBarSet(emotions[0])
    set1 = QBarSet(emotions[1])
    set2 = QBarSet(emotions[2])
    set3 = QBarSet(emotions[3])
    set4 = QBarSet(emotions[4])
    
    set0.append(predY[0][0])
    set1.append(predY[0][1])
    set2.append(predY[0][2])
    set3.append(predY[0][3])
    set4.append(predY[0][4])

    series = QBarSeries()
    series.append(set0)
    series.append(set1)
    series.append(set2)
    series.append(set3)
    series.append(set4)

    chart = QChart()
    chart.setTitle('Sesin analizine göre yüzdelikler')
    chart.addSeries(series)

    chart.setAnimationOptions(QChart.SeriesAnimations)

    axisX = QBarCategoryAxis()
    axisX.append(" ")
    chart.addAxis(axisX, Qt.AlignBottom)
    series.attachAxis(axisX)
    
    axisY = QValueAxis()
    chart.addAxis(axisY, Qt.AlignLeft)
    series.attachAxis(axisY)

    chart.legend().setVisible(True)
    chart.legend().setAlignment(Qt.AlignBottom)
    
    ui4.analiz_gbox.setStyleSheet("padding:5px")
    chartView = QChartView(chart)
    
    vbox = QVBoxLayout()
    ui4.analiz_gbox.setLayout(vbox)
    vbox.addWidget(chartView)

In [17]:
# Seslerin özelliklerinin çıkarılması için tanımlananan fonksiyon
def extract_feature(file_name):
    result=np.array([])
    x, sample_rate = librosa.load(os.path.join(file_name),duration=3)
    x = nr.reduce_noise(audio_clip=x, noise_clip=x)
    mfccs=np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfccs))
    return result

In [18]:
# Gelen sayıyı yüzdeliğe çevirmek için tanımlanan fonksiyon
def toPercantage(number):
    percentage = "{:.2%}".format(number)
    return percentage

In [19]:
# Analizleri kaydetme ekranımızın açılması için tanımlanan fonksiyon
def openSD(): # Save Dialog
    global save_dialog,ui6
    save_dialog = QDialog()
    ui6 = Ui_KaydetDialog()
    ui6.setupUi(save_dialog)
    save_dialog.show()
    ui6.kaydet_btn.clicked.connect(saveFiles)

In [20]:
# Üstteki ekranda tanımlı olan kaydet butonu için tanımlanan fonksiyon
# Bu kısımda bir adet wav uzantılı ses dosyası ve bir adet analizlere sahip txt
# uzantılı metin dosyası oluşturulur
def saveFiles():
    global ui6,isRecPart,myrecording,texts,predY,emotions,file_path
    
    parent_dir = os.getcwd()
    child_dir = "Analyzes"
    
    path=os.path.join(parent_dir,child_dir)
    if not os.path.exists(path):
        print("not exist")
        os.mkdir(path)
    
    filename = ui6.filename_LE.text()
    realEmotion = ui6.realemo_LE.text()
    
    path=os.path.join(path,filename)
    
    sname = filename+".wav"
    tname = filename+".txt"
    
    now = datetime.datetime.now()
    current_time = now.strftime("%H:%M:%S")
    
    if not os.path.exists(path):
        os.mkdir(path)
    else:
        path = findNotExistName(path)
        os.mkdir(path)
    
    answers = [sname,current_time,realEmotion,"MLP"," "]
    
    pathS=os.path.join(path,sname)
    pathT=os.path.join(path,tname)
    
    if isRecPart:
        write(pathS, 44100, myrecording)  # Save as WAV file 
    else:
        copyfile(file_path, pathS)
        
    textF = open(pathT,"w+")
    for text, answer in zip(texts, answers):
        textF.write(text+answer+"\n")
    for prob, emotion in zip(predY[0], emotions):
        textF.write(emotion+" => "+toPercantage(prob)+"\n")
        
    textF.close() 
    save_dialog.hide()

In [21]:
# Klasör ismini kontrol etmek için tanımlı fonksiyon
# Aynı isme sahip klasör var ise uygun bir klasör ismi aramaya yaramakta
def findNotExistName(path):
    i = 1
    while True:
        if not os.path.exists(path+"-"+str(i)):
            return path+"-"+str(i)
        else:
            i=i+1

In [22]:
# Tıklanma olaylarını tanımladık
ui.mic_btn.clicked.connect(openRW)
ui.file_btn.clicked.connect(openFileWindow)
ui2.record_btn.clicked.connect(RPP)
ui2.delete_btn.clicked.connect(deleteSound)
ui2.okey_btn.clicked.connect(lambda: openLW())
mainWindow.show()
# En son uygulamayı sonlandırırız
sys.exit(app.exec_())

C:/Users/Omer/Desktop/Dersler/Proje/Emotion_Recognition/test/test1.wav
[[5.65056932e-05 1.37764965e-02 9.83558873e-01 2.75572189e-05
  2.58056809e-03]]


SystemExit: 0

C:\Users\Omer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
